<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# DSI-13 Capstone Project: 
# Recommender Systems

_Authors: Davis Hong_

---

### Problem Statement:
A Recommender System refers to a system that is capable of predicting the future preference of a set of items for a user, and recommend somthing s/he likes (content based) or something preferred by other users with similar tastes (collaborative). In the past where brick and mortar is the norm, recommendations are usually made through word of mouth. However, with the internet, availability of large amount of data and fast computing power, recommendations not only can be done real-time but also fairly accurate. Recommendations are very common nowadays to the point, it has becoming 'in-your-face'. Businesses especially those relying heavily or solely on the internet for revenue, recommender systems will greatly boost the growth or even the very survival of the business.

There are many type of recommender systems but this project, we will focus on developing a recommender system to recommend movies to users based on a hybrid model;

<a id="two-classical-recommendation-methods"></a>
### Two classical recommendation methods

- **Collaborative Filtering**: _(similar people)_
    - If you like the same 5 movies as someone else, you'll likely enjoy other movies they like.
    - There are two main types: (a) Find users who are similar and recommend what they like (**user-based**), or (b) recommend items that are similar to already-liked items (**item-based**).
   

- **Content-Based Filtering** _(similar items)_
    - If you enjoy certain characteristics of movies (e.g. certain actors, genre, etc.), you'll enjoy other movies with those characteristics.
    - Note this can easily be done using machine learning methods! Each movie can be decomposed into features. Then, for each user we compute a model -- the target can be a binary classifier (e.g. "LIKE"/"DISLIKE") or regression (e.g. star rating).
<br>
The initial model is based on a simple non-machine learning model and the results will be compared with a machine learning based model. The machine learning model will be expected to produce better results.

### Data Source:
Data source: The 1M Movie Lens datasets will be used for the project.

Datasets can be found at https://grouplens.org/datasets/movielens/1m/

Data dictionary: http://files.grouplens.org/datasets/movielens/ml-1m-README.txt


### Potential challenges and obstacles:
As there are no metrics to measure whether the recommender engine is good or not thus some form of human interpretation of the results will be required to gauge the performance of models.

### Measuring of results.
Whenever possible, baseline score will be set and metrics such as MSE, RSME, etc will be used to measure the effectiveness of the model.

Bearing any unforeseen circumstances for eg, interruptions cause by COVID19, this project should be achievable as data is available. Data cleaning is expected but minimal.

### Let's get started...

### Load required libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn import preprocessing
sns.set()
sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Load datasets
---

We'll be using the [MovieLens](https://grouplens.org/datasets/movielens/) dataset for building our recommendation engine. There are 3 datasets namely movies, ratings and users which will be used to build the recommender system. 

In [2]:
pwd

'P:\\GitHub\\General-Assembly-Projects\\Capstone'

In [3]:
movies = pd.read_csv('.\datasets\\movies.csv')
ratings = pd.read_csv('.\datasets\\ratings.csv')
users = pd.read_csv('.\datasets\\users.csv')

In [4]:
print(f'Shape of Movies dataframe: {movies.shape}\n')
print(f'Any null values in the dataframe:\n{movies.isnull().sum()}\n')
print(f'Data types:\n{movies.dtypes}')
movies.head()

Shape of Movies dataframe: (3883, 3)

Any null values in the dataframe:
movieid    0
title      0
genres     0
dtype: int64

Data types:
movieid     int64
title      object
genres     object
dtype: object


,movieid,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
print(f'Shape of Ratings dataframe: {ratings.shape}\n')
print(f'Any null values in the dataframe:\n{ratings.isnull().sum()}\n')
print(f'Data types:\n{ratings.dtypes}')
ratings.head()

Shape of Ratings dataframe: (1000209, 4)

Any null values in the dataframe:
userid       0
movieid      0
rating       0
timestamp    0
dtype: int64

Data types:
userid       int64
movieid      int64
rating       int64
timestamp    int64
dtype: object


,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
print(f'Shape of Users dataframe: {users.shape}\n')
print(f'Any null values in the dataframe:\n{users.isnull().sum()}\n')
print(f'Data types:\n{users.dtypes}')
users.head()

Shape of Users dataframe: (6040, 5)

Any null values in the dataframe:
userid        0
gender        0
age           0
occupation    0
zipcode       0
dtype: int64

Data types:
userid         int64
gender        object
age            int64
occupation     int64
zipcode       object
dtype: object


,userid,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [7]:
# Per data dictionary, there are additional information on the numeric codes 
# used in the users dataset. 

#Create dictionary for age group.
data = {'age': [1, 18, 25, 35, 45, 50, 56], 
        'agegrp': ['Under 18', '18-24', '25-34', '35-44', '45-49', '50-55', '56+']} 

# Create DataFrame for age group
age_grp = pd.DataFrame(data)

# Create dictionary for occupation
data = {'code': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 
        'occupation': ['other', 'academic/educator', 'artist', 'cleric/admin',
                       'college/grad student', 'customer service',
                       'doctor/health care', 'executive/managerial', 'farmer',
                       'homemaker', 'K-12 student', 'lawyer', 'programmer',
                      'retired', 'sales/marketing', 'scientist',
                      'self-employed', 'technician/engineer',
                      'tradesman/craftsman', 'unemployed', 'writer']} 

# Create DataFrame for age group
occupation = pd.DataFrame(data)

In [8]:
age_grp

,age,agegrp
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49
5,50,50-55
6,56,56+


In [9]:
occupation

,code,occupation
0,0,other
1,1,academic/educator
2,2,artist
3,3,cleric/admin
4,4,college/grad student
5,5,customer service
6,6,doctor/health care
7,7,executive/managerial
8,8,farmer
9,9,homemaker


## Drop unnecessary columns
---

We won't need the `timestamp` column from `ratings`, nor will we need the `genres` column from `movies`. Drop both columns in the cells below.

In [10]:
movies.drop('genres', axis = 1, inplace = True)
ratings.drop('timestamp', inplace = True, axis = 1)

## Merge `movies` and `ratings`
---

Use `pd.merge` to **inner join** `movies` with `ratings` on the `movieId` column.

In [11]:
movie_ratings = pd.merge(ratings, movies, on = 'movieid')

In [12]:
# Print first and last 5 rows of the merged dataframe
print(movie_ratings.head())
print(movie_ratings.tail())

   userid  movieid  rating                                   title
0       1     1193       5  One Flew Over the Cuckoo's Nest (1975)
1       2     1193       5  One Flew Over the Cuckoo's Nest (1975)
2      12     1193       4  One Flew Over the Cuckoo's Nest (1975)
3      15     1193       4  One Flew Over the Cuckoo's Nest (1975)
4      17     1193       5  One Flew Over the Cuckoo's Nest (1975)
         userid  movieid  rating                                        title
1000204    5949     2198       5                           Modulations (1998)
1000205    5675     2703       3                        Broken Vessels (1998)
1000206    5780     2845       1                            White Boys (1999)
1000207    5851     3607       5                     One Little Indian (1973)
1000208    5938     2909       4  Five Wives, Three Secretaries and Me (1998)


## Create pivot table
---

Because we're creating an item-based collaborative recommender (where item in this case is our movies), we'll set up our pivot table as follows:
1. The `title` will be the index
2. The `userId` will be the column
3. The `rating` will be the value

**If we were building a user-based collaborative recommender, what would change about this pivot table?**

In [13]:
pivot = pd.pivot_table(movie_ratings, index = 'title', columns='userid', values = 'rating')
pivot.head()

userid,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
title,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Night Mother (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...And Justice for All (1979),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create sparse matrix
Calculate the cosine similarity for each movie using the `pairwise_distances` function. Before that, we need to create a sparse matrix (datatype) using `scipy`'s `sparse` module.

In [14]:
sparse_pivot = sparse.csr_matrix(pivot.fillna(0))

In [15]:
type(sparse_pivot)

scipy.sparse.csr.csr_matrix

## Calculate cosine similarity
Use `sklearn's` built-in `pairwise_distances` function for the recommender. It will return a square matrix, comparing every movie with every other movie in the dataset.

In [16]:
# Create a distance metrics similar to    
recommender = pairwise_distances(sparse_pivot, metric = 'cosine')

In [17]:
recommender_df = pd.DataFrame(recommender, columns = pivot.index, index = pivot.index)

In [18]:
recommender_df.head()

title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",0.000000,0.927643,0.962989,0.920709,0.939162,1.00000,0.941381,0.810035,0.827746,0.905215,...,0.961275,0.923526,1.000000,0.955926,1.0,0.954720,0.960605,1.000000,0.879758,0.972997
'Night Mother (1986),0.927643,0.000000,0.884710,0.884455,0.840474,1.00000,0.923202,0.852563,0.904078,0.888587,...,0.946990,0.912172,0.936242,0.864038,1.0,0.908850,0.925213,1.000000,1.000000,0.922193
'Til There Was You (1997),0.962989,0.884710,0.000000,0.901244,0.933699,0.91975,0.872105,0.887346,0.874330,0.920885,...,0.970800,0.937107,1.000000,0.920813,1.0,0.977406,0.920739,1.000000,0.952474,0.936716
"'burbs, The (1989)",0.920709,0.884455,0.901244,0.000000,0.856380,1.00000,0.807809,0.753073,0.824115,0.829281,...,0.886614,0.792103,0.980038,0.861936,1.0,0.944296,0.838826,1.000000,0.966433,0.889475
...And Justice for All (1979),0.939162,0.840474,0.933699,0.856380,0.000000,1.00000,0.924907,0.805846,0.883621,0.794514,...,0.910002,0.846994,0.932991,0.890971,1.0,0.913920,0.889133,0.925683,1.000000,0.888960


## Create distances DataFrame
---

At this point, we essentially have a recommender. We'll load it into a `pandas` DataFrame for readability. 

You'll notice that each movie has a "distance" of 0 with itself (along the diagonal).

## Evaluate recommender performance
---

Now comes the fun part! Let's check out a few movies to see if the recommender aligns with our intuition. In the cell below we'll do the following:
1. Create a search term
2. Use that to find all titles matching the search query
3. For each title, we'll list off the following:
  1. The average rating
  2. The number of ratings
  3. The ten most similar movies

In [19]:
search = input("What is your favorite movie: ").title()

What is your favorite movie: party


In [20]:
for title in movies.loc[movies['title'].str.contains(search), 'title']:
    print(title)
    print(f'Average rating: {pivot.loc[title, :].mean()}') # Average ratings from users
    print(f'Number of rating: {pivot.loc[title, :].count()}')
    print('')
    print('Here are some movies with similar movie title and genre as per your input.')
    print('')
    print(recommender_df[title].sort_values()[1:11])
    print('')
    print('*' * 70)
    

Party Girl (1995)
Average rating: 3.238095238095238
Number of rating: 105

Here are some movies with similar movie title and genre as per your input.

title
House of Yes, The (1997)           0.670368
Waiting for Guffman (1996)         0.750168
Clockwatchers (1997)               0.758524
Welcome to the Dollhouse (1995)    0.762261
Dazed and Confused (1993)          0.764377
Opposite of Sex, The (1998)        0.765452
Rushmore (1998)                    0.771404
Muriel's Wedding (1994)            0.771521
Spanking the Monkey (1994)         0.775614
Swimming with Sharks (1995)        0.776030
Name: Party Girl (1995), dtype: float64

**********************************************************************
House Party 3 (1994)
Average rating: 1.736842105263158
Number of rating: 38

Here are some movies with similar movie title and genre as per your input.

title
House Party 2 (1991)                         0.436174
Rhyme & Reason (1997)                        0.668958
House Party (1990)      